In [42]:
import h5py
import os
import cv2
import numpy as np
import scipy.spatial.distance
import scipy.io

from skimage import io 
from sklearn.cluster import KMeans

In [43]:
File_path = '/home/cse/Desktop/Raj_M.Tech_Project/Approach4/HS_Results/Together/location_appended_images'
HS_files = os.listdir(File_path)
print("HS_files", HS_files)
save_path = '/home/cse/Desktop/Raj_M.Tech_Project/Approach4/HS_Results/Together'

HS_files ['0006.mat', '0002.mat', '0001.mat']


In [44]:
def create_dict(temp_dict, dict_size):
# dict_size = 10
  # temp_dict = {}
  for i in range(dict_size):
    temp_dict.update({i: list()})
  return temp_dict

In [45]:
number_of_clusters = 3

In [46]:
dict_size = 3
pixel_values = create_dict({}, dict_size)
cluster_means = create_dict({}, dict_size)
mean_list = []

In [47]:
print('pixel values', pixel_values)
print('cluster means', cluster_means)

pixel values {0: [], 1: [], 2: []}
cluster means {0: [], 1: [], 2: []}


In [48]:
def mkdir(dir_path):
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

In [49]:
im_labelled_images_path = os.path.join(save_path, 'location_appended_results')
mkdir(im_labelled_images_path)

In [50]:
num = 0
for file in HS_files:
  input_img = os.path.join(File_path, file)
  # print("Reading File[%d/%d]: %s" % (num, len(HS_files), input_img))
  # print("file name", file)
  mat_dict = scipy.io.loadmat(input_img)
  for key in mat_dict:
    if type(mat_dict[key]) is np.ndarray:
      image_data = mat_dict[key]  #type: np.ndarray
  
  print("image data shape before axis move:", image_data.shape)
  # image_data = np.moveaxis(image_data, 0, 2)
  # image_data = np.moveaxis(image_data, 0, 1)

  # print("image data after axis move:", image_data.shape)

  img = image_data.reshape((-1, 83))
  img = np.float32(img)
  criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)
  ret, labels, centers = cv2.kmeans(img, number_of_clusters, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
  labels = labels.flatten()
  labels = labels.reshape(1024, 768)
  file = file.rsplit('.', 1)

  for i in range(labels.shape[0]):
    for j in range(labels.shape[1]):
      label = labels[i][j]
      pixel_values[label].append(image_data[i][j][:])

  for label in pixel_values:
    channel_mean = np.mean(pixel_values[label], axis = 0)
    cluster_means[label].append(channel_mean)

  labels = cv2.rotate(labels, cv2.ROTATE_90_CLOCKWISE)
  labels = cv2.flip(labels, 1)
  save_im_labelled_images = im_labelled_images_path + '/' + file[0] + '.png'
  cv2.imwrite(save_im_labelled_images, labels)

  mean_list.append(cluster_means)
  pixel_values = dict((k, list()) for k in pixel_values)
  cluster_means = dict((k, list()) for k in cluster_means)

print('!!!!!! Cluster Mean Calculated Successfully !!!!!!')

image data shape before axis move: (1024, 768, 83)
image data shape before axis move: (1024, 768, 83)
image data shape before axis move: (1024, 768, 83)
!!!!!! Cluster Mean Calculated Successfully !!!!!!


In [51]:
print(mean_list)

[{0: [array([2.44446578e-01, 2.41821953e-01, 2.50947324e-01, 2.47738986e-01,
       2.74771562e-01, 2.90052062e-01, 3.01413723e-01, 3.07600855e-01,
       3.09181142e-01, 3.16978188e-01, 3.25164050e-01, 3.05667764e-01,
       3.05998002e-01, 2.85679098e-01, 2.61381655e-01, 2.33849318e-01,
       2.33142766e-01, 2.30310533e-01, 2.29789518e-01, 2.38992490e-01,
       2.58895832e-01, 2.78575074e-01, 2.76416775e-01, 2.89311014e-01,
       3.12723686e-01, 3.14893079e-01, 3.21814227e-01, 3.35689111e-01,
       3.32966991e-01, 3.44186513e-01, 3.61465307e-01, 3.82498636e-01,
       3.93790411e-01, 3.92462516e-01, 4.01923917e-01, 4.13204619e-01,
       4.13169576e-01, 4.06257863e-01, 4.02346951e-01, 3.97075726e-01,
       3.95330179e-01, 3.97697787e-01, 3.82682655e-01, 3.57138785e-01,
       3.69099280e-01, 3.91039807e-01, 4.00285306e-01, 3.97045042e-01,
       3.97856945e-01, 3.99215623e-01, 3.74498179e-01, 3.62584033e-01,
       3.64252086e-01, 3.69904316e-01, 3.49460111e-01, 3.31378028e-01,


In [52]:
print("length of meanlist", len(mean_list))

length of meanlist 3


In [53]:
print(cluster_means)

{0: [], 1: [], 2: []}


In [54]:
dist = 0
clus_dist = []
euc_dist = []
for i in range(len(mean_list)):
  for j in range(len(mean_list[i])):
    for k in range(len(mean_list[i])):
      # print('i-j', type(mean_list[i][j][0]))
      # print('i-k', type(mean_list[i][k][0]))

      dist = dist + scipy.spatial.distance.euclidean(mean_list[i][j][:], mean_list[i][k][:])
      # dist += np.linalg.norm(mean_list[i][j][0] - mean_list[i][k][0])
    clus_dist.append(dist)
    # print("cluster distance",  len(clus_dist))
    dist = 0

  euc_dist.append(clus_dist)
  clus_dist = []

print('!!!!!! Euclidean Distance Calculated Successfully !!!!!!')

cluster distance 1
cluster distance 2
cluster distance 3
cluster distance 1
cluster distance 2
cluster distance 3
cluster distance 1
cluster distance 2
cluster distance 3
!!!!!! Euclidean Distance Calculated Successfully !!!!!!


In [55]:
print(euc_dist)

[[946.6260122332963, 947.0554337347166, 1092.4956518727813], [946.7373126966161, 946.9189767992024, 1092.4906889836934], [1092.4901533538616, 946.91852073989, 946.7379404994826]]


In [57]:
# mean_list_ind = 0
euc_dist_ind = 0
clusters_ind = -1

File_path_labelled_images = '/home/cse/Desktop/Raj_M.Tech_Project/Approach4/HS_Results/Together/location_appended_results'
HS_files_labelled_images = sorted(os.listdir(File_path_labelled_images))
print("HS_files_labelled_images", HS_files_labelled_images)

save_path = '/home/cse/Desktop/Raj_M.Tech_Project/Approach4/HS_Results/Together'
im_final_segmented_images = os.path.join(save_path, 'Location_Appended_Final_Images')
mkdir(im_final_segmented_images)

for labelled_images in HS_files_labelled_images:
  file = labelled_images.rsplit('.', 1)
  input_labelled_images = os.path.join(File_path_labelled_images, labelled_images)
  img = cv2.imread(input_labelled_images)
  # print('Image before channel removal', img.shape)
  img = np.max(img, axis = 2)
  # print('Image after channel removal', img.shape)
  # print('Image shape[0]', img.shape[0])
  # print('Image shape[1]', img.shape[1])
  
  clusters_ind += 1
  for clusters in range(clusters_ind, len(mean_list), 1):
    for label, mean in mean_list[clusters].items():
      if np.mean(mean[0]) != 0:
        for i in range(img.shape[0]):
          for j in range(img.shape[1]):
            # print(img[i][j])
            if label == img[i][j]:
              # print(1)
            #   print(euc_dist[clusters])
              img[i][j] = euc_dist[clusters][euc_dist_ind]
        euc_dist_ind += 1
        # print('If euc_dist_ind', euc_dist_ind)
      
      else:
        euc_dist_ind += 1
        # print('Else euc_dist_ind', euc_dist_ind)
        continue
    euc_dist_ind = 0
  
    # clusters += 1
    # img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
    # img = cv2.flip(img, 1)
    
    max_pixel_val = np.max(img)
    img = img/max_pixel_val
    img = img * 255

    # kernel = np.ones((9, 9), np.uint8)
    # img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)

    save_im_final_featureMap = im_final_segmented_images + '/' + file[0] + '.png'
    cv2.imwrite(save_im_final_featureMap, img)
    break

  # save_im_fdict_size = 5inal_featureMap = im_final_segmented_images + '/' + file[0] + '.png'
  # cv2.imwrite(save_im_final_featureMap, img)

print('!!!!!!!!! Image saved successfully !!!!!!!!!!')

HS_files_labelled_images ['0001.png', '0002.png', '0006.png']
!!!!!!!!! Image saved successfully !!!!!!!!!!
